# Activity 9

This activity tests your skills
in approaching an engineering problem
as solving a system of nonlinear algebraic equations.

## Scenario

Referring to [the diagram](./act-09.png),
a device has an active power consumption $P$
when operating at a lagging power factor $\Phi$.
The requisite power is conveyed by the
resistance $R$,
inductance $L$
and the capacitor $C$,
from a source
configured to provide a steady RMS voltage $U$
at a steady frequency $F$.

You are to solve for the following quantities.
- $V$, the RMS operating voltage of the device
- $\delta_{\text{l}}$,
  the phase angle of the voltage phasor at which the device receives power
- $\delta_{\text{s}}$
  the phase angle of the voltage phasor provided by the source

## Modelling

Cast the original task as
solving a system of nonlinear algebraic equations
in three unknowns
(namely, $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$).
Introduce a vector $\boldsymbol{x}$ 
to collect the unknowns in the order presented above,
so that the system of equations can be written in vector form as
$\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$.
Set the voltage phasor across the input terminals of the load
to have the reference phase angle.

## Core code

In [1]:
import math as mt

import numpy as np
import scipy as sp

Define a Python function
`f()` that takes

- one positional argument,
  `x`, representing $\boldsymbol{x}$;
  and
- four keyword arguments,
  `U`, `F`, `R`, `L`, `C`, `P`, and `Phi`
  representing $U$, $F$, $R$, $L$, $C$, $P$, and $\Phi$ respectively.

`f()` returns a NumPy array representing
$\boldsymbol{f}\!\left(\boldsymbol{x}\right)$.

Moreover, define a Python function `dfdx()`
that takes the same arguments as `f()`,
and returns a NumPy array representing
the Jacobian of $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ w.r.t. $\boldsymbol{x}$.


In [2]:
def f(x, U, F, R, L, C, P, Phi):
    V, dl, ds = x
    Pw = P * 10**-3
    omega = 2 * mt.pi * F
    XL = omega * L
    YC = omega * C
    Q_term = Pw * mt.tan(mt.acos(Phi))

    f1 = R*(Pw/V) - (XL * (Q_term - (V**2)*YC) / V) - U*mt.cos(ds) + V*mt.cos(dl)
    f2 = (R/V)*(Q_term - (V**2)*YC) + (XL*Pw)/V - U*mt.sin(ds) + V*mt.sin(dl)
    f3 = dl
    
    return np.array([f1, f2, f3])

def dfdx(x, U, F, R, L, C, P, Phi):
    V, dl, ds = x
    Pw = P * 10**-3
    omega = 2 * mt.pi * F
    XL = omega * L
    YC = omega * C
    Q_term = Pw * mt.tan(mt.acos(Phi))

    df1_dV = -R*Pw/(V**2) + XL*Q_term/(V**2) + XL*YC + mt.cos(dl)
    df2_dV = -R*Q_term/(V**2) - R*YC - XL*Pw/(V**2) + mt.sin(dl)
    
    jac = np.array([
        [df1_dV, -V*mt.sin(dl), U*mt.sin(ds)],
        [df2_dV, V*mt.cos(dl), -U*mt.cos(ds)],
        [0.0, 1.0, 0.0]
    ])
    
    return jac

## Assessment

Every time you solve for $\boldsymbol{x}$,
- define Python variables
  `U`, `F`, `R`, `L`, `C`, `P`, and `PHI`
  for storing the values of parameters
  $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$,
  respectively;
- define a Python variable `x_ini`
  for storing your initial guesses of the variables;
- use [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html),
  particularly its `args` keyword argument;
- define a Python variable `x_out` to store $\boldsymbol{x}$;
- verify if $\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$ is sufficiently satisfied;
- print the value of $V$ in kilovolts;
  and
- print the values of $\delta_{\text{l}}$ and $\delta_{\text{s}}$ in degrees.

### Task 1

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
40 kilowatts, 0.87.

In [3]:
U = 13.8
F = 60
R = 0.21
L = 5.570e-3
C = 1.560e-3
P = 40
PHI = 0.87

x_ini = np.array([U, 0.0, 0.0])

res = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx, method='hybr')
x_out = res.x

V_res, dl_res, ds_res = x_out
residual = f(x_out, U, F, R, L, C, P, PHI)

print(f"Residual f(x): {residual}")
print(f"V: {V_res:.4f} kV")
print(f"delta_l: {mt.degrees(dl_res):.4f} degrees")
print(f"delta_s: {mt.degrees(ds_res):.4f} degrees")

Residual f(x): [-7.10542736e-15  2.45106373e-26  3.97353425e-27]
V: 6.1685 kV
delta_l: 0.0000 degrees
delta_s: -3.1048 degrees


### Task 2

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
32 kilowatts, 0.97.

In [4]:
U = 13.8
F = 50
R = 0.17
L = 6.685e-3
C = 1.872e-3
P = 32
PHI = 0.97

x_ini = np.array([U, 0.0, 0.0])

res = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx, method='hybr')
x_out = res.x

V_res, dl_res, ds_res = x_out
residual = f(x_out, U, F, R, L, C, P, PHI)

print(f"Residual f(x): {residual}")
print(f"V: {V_res:.4f} kV")
print(f"delta_l: {mt.degrees(dl_res):.4f} degrees")
print(f"delta_s: {mt.degrees(ds_res):.4f} degrees")

Residual f(x): [-2.66453526e-15  8.11649662e-28  1.31567803e-28]
V: 6.1691 kV
delta_l: 0.0000 degrees
delta_s: -2.5154 degrees


### Task 3

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
214 kilowatts, 0.87.

In [5]:
U = 32.5
F = 60
R = 0.21
L = 5.570e-3
C = 1.560e-3
P = 214
PHI = 0.87

x_ini = np.array([U, 0.0, 0.0])

res = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx, method='hybr')
x_out = res.x

V_res, dl_res, ds_res = x_out
residual = f(x_out, U, F, R, L, C, P, PHI)

print(f"Residual f(x): {residual}")
print(f"V: {V_res:.4f} kV")
print(f"delta_l: {mt.degrees(dl_res):.4f} degrees")
print(f"delta_s: {mt.degrees(ds_res):.4f} degrees")

Residual f(x): [-1.24344979e-14  4.44089210e-16  3.07600199e-27]
V: 14.5269 kV
delta_l: 0.0000 degrees
delta_s: -3.1068 degrees


### Task 4

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
173 kilowatts, 0.97.

In [6]:
U = 32.5
F = 50
R = 0.17
L = 6.685e-3
C = 1.872e-3
P = 173
PHI = 0.97

x_ini = np.array([U, 0.0, 0.0])

res = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx, method='hybr')
x_out = res.x

V_res, dl_res, ds_res = x_out
residual = f(x_out, U, F, R, L, C, P, PHI)

print(f"Residual f(x): {residual}")
print(f"V: {V_res:.4f} kV")
print(f"delta_l: {mt.degrees(dl_res):.4f} degrees")
print(f"delta_s: {mt.degrees(ds_res):.4f} degrees")

Residual f(x): [-5.32907052e-15 -2.22044605e-16 -3.21567954e-27]
V: 14.5285 kV
delta_l: -0.0000 degrees
delta_s: -2.5166 degrees


### Task 5

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
69.0 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
421 kilowatts, 0.87.

In [7]:
U = 69.0
F = 60
R = 0.21
L = 5.570e-3
C = 1.560e-3
P = 421
PHI = 0.87

x_ini = np.array([U, 0.0, 0.0])

res = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx, method='hybr')
x_out = res.x

V_res, dl_res, ds_res = x_out
residual = f(x_out, U, F, R, L, C, P, PHI)

print(f"Residual f(x): {residual}")
print(f"V: {V_res:.4f} kV")
print(f"delta_l: {mt.degrees(dl_res):.4f} degrees")
print(f"delta_s: {mt.degrees(ds_res):.4f} degrees")

Residual f(x): [ 3.19744231e-14 -1.77635684e-15  8.63126242e-28]
V: 30.8331 kV
delta_l: 0.0000 degrees
delta_s: -3.1384 degrees


### Task 6

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
317 kilowatts, 0.97.

In [8]:
U = 32.5
F = 50
R = 0.17
L = 6.685e-3
C = 1.872e-3
P = 317
PHI = 0.97

x_ini = np.array([U, 0.0, 0.0])

res = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx, method='hybr')
x_out = res.x

V_res, dl_res, ds_res = x_out
residual = f(x_out, U, F, R, L, C, P, PHI)

print(f"Residual f(x): {residual}")
print(f"V: {V_res:.4f} kV")
print(f"delta_l: {mt.degrees(dl_res):.4f} degrees")
print(f"delta_s: {mt.degrees(ds_res):.4f} degrees")

Residual f(x): [-9.94759830e-14  1.11022302e-15  9.49567723e-27]
V: 14.5305 kV
delta_l: 0.0000 degrees
delta_s: -2.4795 degrees


## Instructions

Do not use any library or module other than those pre-imported.

### Scoring

In each Task,
- a correct use of
  [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)
  merits two (2) points;
- obtaining a correct $V$ merits two (3) points;
- obtaining a correct $\delta_{\text{l}}$ merits two (3) points;
- obtaining a correct $\delta_{\text{s}}$ merits two (3) points;
  and
- obtaining $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ close enough to $\boldsymbol{0}$ merits six (9) points.


Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 120 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-09_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-09_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-12-11*